In [1]:
#!/usr/bin/env python3
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import hashlib
import hmac
import math
import sys
import logging
import os
import getopt
import traceback


# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)	# 显示行数
pd.set_option('display.max_columns', 500)	# 显示列数
pd.set_option('display.width', 1000)		# 显示宽度

g_worthDir = '../pkl'  # 
g_dbfile = f"{g_worthDir}/hq.db" # 

g_serverTimeadj = 10
g_locTimeadj = 28800 # 28800 0

g_name = 'backtest'
# curDate = time.strftime("%Y%m%d") # %Y%m%d %H%M%S  # %Y-%m-%d %H:%M:%S
timestamp = int(round((time.time() ) * 1000) )

serverUrl = 'http://testnet.binancefuture.com'
# serverUrl = 'http://www.binance.com'
# serverUrl = 'https://fapi.binance.com'
#  #ation126@hotmail.com
api_key    = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"  # 密钥
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"  # 私钥


recvWindow = '5000'
g_access = 'backtest'

url = f'{serverUrl}/fapi/v1/time'
res = requests.get(url)
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    serverTime = rt['serverTime']
    g_serverTimeadj = ((rt['serverTime']- timestamp) ) //1000

# df_balance = pd.DataFrame()
# df_account = pd.DataFrame()
# df_position = pd.DataFrame()
# df_posirisk = pd.DataFrame()
# df_order = pd.DataFrame()
# df_trade = pd.DataFrame()

version = "1.0.1"

print(f"{sys.version} , {sys.executable} , pandas: {pd.__version__}")
print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****import**v{version}*** ')

# print( f'localtime :{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} ,\n time/serverTime : {time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(serverTime//1000+g_locTimeadj))} ,\n datetime/serverTime : {datetime.fromtimestamp(serverTime//1000+g_locTimeadj)} ,\n {serverTime= } , {g_locTimeadj= } *****import**v{version}*** ')


3.10.4 (main, Nov 29 2022, 20:00:25) [GCC 9.4.0] , /home/codespace/.python/current/bin/python3 , pandas: 1.5.2
2022-12-29 10:39:44 *****import**v1.0.1*** 


In [2]:
###日志 logger##20220825A######
def make_logger(name, log_level=logging.INFO, log_file="log.txt", file_mode="a"): #w写 a追加
    formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s') # .%(msecs)03d
    
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(level=log_level)
        handler = logging.FileHandler(log_file, mode=file_mode)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)

        handler = logging.StreamHandler(sys.stdout)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

logger = make_logger(g_name, log_level=logging.DEBUG, log_file= f"../logs/{g_name if ('g_name' in dir() ) else 'test' }_{ time.strftime('%Y%m%d') }.log")

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****logging***** ')

2022-12-29 10:39:44 *****logging***** 


In [3]:
###服务器时间 QryServerTime##20220919A######
def QryServerTime():
    url = f'{serverUrl}/fapi/v1/time'
    res = requests.get(url)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else:
        srvTime = rt['serverTime']  
    
    return srvTime  
    
print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****QryServerTime***** ')

2022-12-29 10:39:44 *****QryServerTime***** 


In [4]:
###最新价格 ticker/price###get###QryTcikerPrice###
def QryTcikerPrice(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/ticker/price'

    params = {"symbol":symbol, }

    res = requests.get(url, params=params) #
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')  
    else :
        logger.info(f'res : {res.text}')  #{"symbol":"BTCUSDT","price":"20137.10","time":1661961001841}

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****最新价格 ticker/price###get###QryTcikerPrice***** ') 

2022-12-29 10:39:44 *****最新价格 ticker/price###get###QryTcikerPrice***** 


In [5]:
###最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex###
def QryPremiumIndex(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/premiumIndex'

    params = {"symbol":symbol, }

    res = requests.get(url, params=params) #
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')
    else :
        logger.info(f'res : {res.text}')  
        # {"symbol":"BTCUSDT","markPrice":"20133.32973555","indexPrice":"20133.75155317","estimatedSettlePrice":"20056.77850121","lastFundingRate":"-0.00194053","interestRate":"0.00010000","nextFundingTime":1661961600000,"time":1661961098000}

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex***** ') 

2022-12-29 10:39:44 *****最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex***** 


In [6]:
###深度信息 depth###get###QryDepth###
def QryDepth(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/depth'

    params = {"symbol":symbol, "limit":5, } 

    res = requests.get(url, params=params)  
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')
    else :
        logger.info(f'res : {res.text}')  
        # {"lastUpdateId":27532317617,"E":1664272323243,"T":1664272323241,"bids":[["20173.30","4.937"],["20173.00","0.001"],["20172.90","4.228"],["20172.50","18.436"],["20171.30","14.832"]],"asks":[["20183.40","1.430"],["20183.80","61.862"],["20184.20","128.430"],["20184.60","71.158"],["20185.00","3.459"]]}

        print(f"rt : {rt}")
        print(f"rt['bids'][0][0] : {rt['bids'][0][0]}")
        print(f"rt['bids'][0][0] : {rt['bids'][0][0]} , rt['asks'][0][0] : {rt['asks'][0][0]}")

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****###深度信息 depth###get###QryDepth###***** ') 

Now: 1.0.1 : 2022-12-29 02:39:44 *****###深度信息 depth###get###QryDepth###***** 


In [7]:
###查询行情 kline###continuousKlines###
def continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m', startTime=0, endTime=0, limit=499 ):
    
    global serverUrl
    url = f'{serverUrl}/fapi/v1/continuousKlines'

    # pair = 'btcusdt'
    # contractType = 'PERPETUAL'
    # interval = '5m' #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    # startTime = 0
    # endTime = 0
    # limit = 3 #499

    if limit < 0 or startTime < 0 or endTime < 0 or (startTime > endTime):
        k_df = pd.DataFrame()
        logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" or "startTime" or "endTime" is not valid."}}')
        return k_df

    elif limit == 0:
        if startTime == 0 and endTime == 0:
            k_df = pd.DataFrame()
            logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" is not valid."}}') 
            return k_df

        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime }  
    elif limit > 0:
        if startTime == 0 and endTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "limit":limit, }  
        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "limit":limit, }   
        elif startTime > 0 and endTime > 0:  # startTime + limit < endTime
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "endTime":endTime, "limit":limit, }   

    logger.info(f'{params=}')
    
    res = requests.get(url, params=params)

    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        k_df = pd.DataFrame()
        logger.error(f'res : {res.text}')
    else:
        k_df = pd.DataFrame(rt)
        if k_df.empty:
            return k_df    
        
        k_df.columns = ['openTime','open','high','low','close','vol','closeTime','Amt','numTrade','bidVol','bidAmt','ignore'] #
        
        k_df['open']  = pd.to_numeric(k_df['open'] , errors='coerce', downcast='float')
        k_df['high']  = pd.to_numeric(k_df['high'] , errors='coerce', downcast='float')
        k_df['low']  = pd.to_numeric(k_df['low'] , errors='coerce', downcast='float')
        k_df['close']  = pd.to_numeric(k_df['close'] , errors='coerce', downcast='float')
        k_df['vol']  = pd.to_numeric(k_df['vol'] , errors='coerce', downcast='float')
        k_df['Amt']  = pd.to_numeric(k_df['Amt'] , errors='coerce', downcast='float')
        k_df['bidVol']  = pd.to_numeric(k_df['bidVol'] , errors='coerce', downcast='float')
        k_df['bidAmt']  = pd.to_numeric(k_df['bidAmt'] , errors='coerce', downcast='float')
        k_df['openTime']  = pd.to_numeric(k_df['openTime'] , errors='coerce', downcast='integer')
        k_df['closeTime']  = pd.to_numeric(k_df['closeTime'] , errors='coerce', downcast='integer')
        k_df['numTrade']  = pd.to_numeric(k_df['numTrade'] , errors='coerce', downcast='integer')

        k_df['openDateTime'] = pd.to_datetime(k_df['openTime'], origin="1970-01-01 08:00:00", unit="ms") #, origin="1970-01-01 08:00:00" , unit="ms"  #debug
        k_df['closeDateTime'] = pd.to_datetime(k_df['closeTime']//1000, origin="1970-01-01 08:00:00", unit="s") 
        # k_df['time'] = pd.to_datetime(int(time.time()), unit='s', unorigin="1970-01-01 08:00:00")
        
        k_df.drop(['ignore'], axis=1, inplace=True)

        k_df.sort_index(axis=1, ascending=True, inplace=True)

    return k_df

#权重 [1,100)  1 ; [100, 500)	2 ; [500, 1000]	5 ;  > 1000	10 ;

# [
# 1661744100000,      // 开盘时间 openTime
# "18879.99",         // 开盘价 open
# "18900.00",         // 最高价 high
# "18878.98",         // 最低价 low
# "18896.13",         // 收盘价(当前K线未结束的即为最新价) close
# "492.363",          // 成交量 vol
# 1661744399999,      // 收盘时间 closeTime
# "9302145.66080",    // 成交额 Amt
# 1874,               // 成交笔数 numTrade
# "385.983",          // 主动买入成交量 bidVol
# "7292402.33267",    // 主动买入成交额 bidAmt
# "0"                 // 请忽略该参数 ignore
# ]

# k_df = continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m',limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(limit=2, endTime =1661824559999 ) # 1661824559999 ,2 
# k_df = continuousKlines(limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(startTime =1661824259000, endTime =1661824559999 ) # 5
# k_df = continuousKlines(limit=2, startTime =1661824259000, ) # 2
# k_df = continuousKlines(limit=0, startTime =0, endTime =0 ) #err 0 
# k_df = continuousKlines(limit=-2 ) # err 0
# k_df = continuousKlines(limit=2 ) # 2
# print(f'{k_df}')  #11s

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****查询行情 kline###continuousKlines***** ')

2022-12-29 10:39:45 *****查询行情 kline###continuousKlines***** 


In [8]:
###查询历史行情 kline###histUKline###
def histUKline(symbol="BTCUSDT", interval="1m", histDays=1, limit=998):
    global g_locTimeadj

    intervalCoef = {'m':60, 'h':60*60, 'd': 60*60*24, 'w':60*60*24*7, 'M':60*60*24*30}
    intervalSec = int(interval[0:-1]) * intervalCoef[interval[-1] ]
    curSec = int(time.time())
    startSec = int(time.mktime(time.strptime(time.strftime("%Y%m%d", time.localtime(time.time()-60*60*24*histDays ) ), '%Y%m%d')))
    startSec = startSec -g_locTimeadj -1

    interval_pre = interval
    if interval_pre == '10m':
        interval = '5m'
        intervalSec = intervalSec//2

    totalCount = math.ceil( (curSec - startSec )/intervalSec )
    logger.info(f'{interval=}, {startSec=}, {curSec=}, {intervalSec=}, {limit=}, {intervalSec*limit=}, {totalCount=}, {histDays=}')

    # ukdfHist = pd.DataFrame(columns=['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', ])
    ukdfHist = pd.DataFrame()

    count = 1
    for Sec in range(startSec, curSec, intervalSec*limit):

        print(f'{count=}:', "continuousKlines(", f'{symbol=}', ',', f'{interval=}', ',', f'{limit=}', ',', f'{Sec*1000+999=}', ', endTime=', f'{(Sec+intervalSec*limit-1)*1000+999}' , ' )')

        ukRet = continuousKlines(pair=symbol, interval=interval, limit=limit, startTime=Sec*1000+999, endTime=(Sec+intervalSec*limit-1)*1000+999 )

        kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+g_locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close,'high': rs.high,'low': rs.low,'vol': rs.vol, 'amt': rs.Amt} for _, rs in ukRet.iterrows() ])

        # ukdfHist.append(kdf)
        ukdfHist = pd.concat([ukdfHist,kdf],ignore_index=True)
        del kdf

        count += 1
        time.sleep(1)

    logger.info(f'ukdfHist.iloc[:5,:] :\n{ukdfHist.iloc[:5,:]}')
    logger.info(f'ukdfHist.iloc[-5:,:] :\n{ukdfHist.iloc[-5:,:]}')

    if interval_pre == '10m':
        interval = interval_pre
        intervalSec = int(intervalSec*2)
        rePeriod = '10T'

        ukdfHistCp = ukdfHist.copy()
        ukdfHistCp['dateTime'] = ukdfHistCp['closeSec'].apply(lambda x: datetime.fromtimestamp(x))
        ukdfHistCp = ukdfHistCp.set_index(keys=['dateTime'], drop=True)
        # ukdfHistCp = ukdfHistCp.reindex(ukdfHistCp['dateTime'].sort_values(ascending=True).index)

        logger.info(f'ukdfHistCp.iloc[:5,:] :\n{ukdfHistCp.iloc[:5,:]}')

        if not ukdfHistCp.empty:
            openSr = ukdfHistCp['open'].resample(rePeriod, label='right').first()  
            openTimeSr = ukdfHistCp['openTime'].resample(rePeriod, label='right').first()  
            closeSr = ukdfHistCp['close'].resample(rePeriod, label='right').last()  # resample
            closeTimeSr = ukdfHistCp['closeTime'].resample(rePeriod, label='right').last()  
            closeSecSr = ukdfHistCp['closeSec'].resample(rePeriod, label='right').last()  
            tradeDateSr = ukdfHistCp['tradeDate'].resample(rePeriod, label='right').last()  
            
            highSr = ukdfHistCp['high'].resample(rePeriod, label='right').max()
            lowSr = ukdfHistCp['low'].resample(rePeriod, label='right').min()
            volSr = ukdfHistCp['vol'].resample(rePeriod, label='right').sum()
            AmtSr = ukdfHistCp['Amt'].resample(rePeriod, label='right').sum()

            ukdf = pd.concat([tradeDateSr, openTimeSr, closeTimeSr, closeSecSr,openSr, closeSr, highSr, lowSr, volSr, AmtSr], axis=1) 
            
        ukdf.columns = ['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', 'high', 'low', 'vol', 'Amt']
        ukdf.reset_index(drop=True,inplace=True)

        if ukdf.iloc[-1]['closeSec'] - ukdf.iloc[-2]['closeSec'] < intervalSec:
            ukdf.drop([len(ukdf)-1],inplace=True)

        # ukdf = ukdfHistCp.copy()
        # logger.info(f'{ukdf.iloc[:5,:]=}')
        
        del ukdfHistCp

    else:
        ukdf = ukdfHist.copy()

    del ukdfHist

    ukdf['close'] = ukdf['close'].astype(float)
    ukdf['pct'] = ukdf['close']/ukdf['close'].shift(1)-1  #涨跌幅
    ukdf.fillna(0, inplace=True)

    logger.info(f'ukdf.iloc[:5,:] :\n{ukdf.iloc[:5,:]}')
    logger.info(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')

    return ukdf

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****查询历史行情 kline###histUKline***** ')

2022-12-29 10:39:45 *****查询历史行情 kline###histUKline***** 


In [9]:
###查询历史行情 日kline###getDayUKline###
def getDayUKline(symbol="BTCUSDT", interval="1m", startDate="0", endDate="0", histDays=0, limit=499):

    if startDate=="0" and endDate=="0" and histDays==0:
        startSec = int(datetime.strptime(datetime.now().strftime("%Y%m%d"), "%Y%m%d").strftime("%s"))
        endSec = int( ( datetime.fromtimestamp(startSec)+ timedelta(days=1) ).strftime("%s") )
        
    elif startDate=="0" and endDate=="0" and abs(histDays)>0:
        endSec = int(datetime.strptime(f"{datetime.now().strftime('%Y%m%d')}235959", '%Y%m%d%H%M%S').strftime("%s"))        
        startSec = int( ( datetime.fromtimestamp(endSec - 60*60*24) - timedelta(days=abs(histDays)) ).strftime("%s") )
        
    elif int(startDate)>=20190901 and int(endDate)>int(startDate):
        startSec = int(datetime.strptime(startDate, "%Y%m%d").strftime("%s"))  
        endSec = int(datetime.strptime(f"{endDate}235959", "%Y%m%d%H%M%S").strftime("%s")) 
        
    elif int(startDate)>=20190901 and endDate=="0" and histDays==0:
        startSec = int(datetime.strptime(startDate, "%Y%m%d").strftime("%s"))  
        endSec = int(datetime.strptime(f"{datetime.now().strftime('%Y%m%d')}235959", '%Y%m%d%H%M%S').strftime("%s")) 
        
    elif int(startDate)>=20190901 and endDate=="0" and histDays>0:
        startSec = int(datetime.strptime(startDate, "%Y%m%d").strftime("%s"))  
        endSec = int( ( datetime.fromtimestamp(startSec) + timedelta(days=histDays) ).strftime("%s") )   
        
    elif startDate=="0" and int(endDate)>=20190901 and abs(histDays)>0:
        endSec = int(datetime.strptime(f"{endDate}235959", "%Y%m%d%H%M%S").strftime("%s"))        
        startSec = int( ( datetime.fromtimestamp(endSec - 60*60*24) - timedelta(days=abs(histDays)) ).strftime("%s") )          

    else:
        startSec = int(datetime.strptime(startDate, "%Y%m%d").strftime("%s"))
        endSec = int( ( datetime.fromtimestamp(startSec)+ timedelta(days=histDays) ).strftime("%s") )

    startSec = startSec -g_locTimeadj -1

    intervalCoef = {'m':60, 'h':60*60, 'd': 60*60*24, 'w':60*60*24*7, 'M':60*60*24*30}
    intervalSec = int(interval[0:-1]) * intervalCoef[interval[-1] ]

    totalCount = math.ceil( (endSec - startSec )/intervalSec )
    logger.info(f'{interval=}, {startSec=}, {endSec=}, {intervalSec=}, {limit=}, {intervalSec*limit=}, {totalCount=}, {histDays=}')

    # ukdfHist = pd.DataFrame(columns=['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', ])
    ukdfHist = pd.DataFrame()

    count = 1
    for Sec in range(startSec, endSec, intervalSec*limit):

        stopSec = min(Sec+intervalSec*limit-1 , endSec-1)

        print(f'{count=}:', "continuousKlines(", f'{symbol=}', ',', f'{interval=}', ',', f'{limit=}', ',', f'{Sec*1000=}', ', endTime=', f'{stopSec*1000+999}' , ' )')

        ukRet = continuousKlines(pair=symbol, interval=interval, limit=limit, startTime=Sec*1000, endTime=stopSec*1000+999 )

        # kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+g_locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close,'high': rs.high,'low': rs.low,'vol': rs.vol, 'amt': rs.Amt} for _, rs in ukRet.iterrows() ])

        # ukdfHist.append(kdf)
        ukdfHist = pd.concat([ukdfHist,ukRet],ignore_index=True)
        # del kdf

        count += 1
        time.sleep(0.5)

    logger.info(f'ukdfHist.iloc[:5,:] :\n{ukdfHist.iloc[:5,:]}')
    logger.info(f'ukdfHist.iloc[-5:,:] :\n{ukdfHist.iloc[-5:,:]}')

    return ukdfHist

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****查询历史行情 日kline###getDayUKline***** ')

2022-12-29 10:39:45 *****查询历史行情 日kline###getDayUKline***** 


In [10]:
###指标 Similarity###getSimilarity###
def getSimilarity(ukdf, wid) -> pd.DataFrame:
    ukdfT = ukdf.pivot(index='closeTime', columns='tradeDate', values='close')
    ukdfAmT = ukdfT.iloc[:wid].copy()
    ukdfAmT.dropna(how='any',inplace=True)
    ukdfAmT.reset_index(drop=True,inplace=True)
    ukdfAmT = ukdfAmT.apply(lambda x: x.astype(float))

    logger.info(f"{ukdfAmT.corr()=}")

    df_corr = ukdfAmT.corr().iloc[:, [-1]]
    df_corr.columns=['corr']
    # df_corr = df_corr.reindex(df_corr['corr'].abs().sort_values(ascending=False).index)
    df_corr = df_corr.reindex(df_corr['corr'].sort_values(ascending=False).index)

    logger.info(f"df_corr: {df_corr}")  # corr: df_corr.iloc[1,0], data: df_corr.index[1]
    return df_corr
print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+g_locTimeadj))} *****指标 Similarity###getSimilarity***** ')

2022-12-29 10:39:45 *****指标 Similarity###getSimilarity***** 


In [ ]:
# QryPremiumIndex() # 最新标记价格和资金费率

In [ ]:
# 1.查询指定日期行情 getDayUKline() 30m
g_symbol = "BTCUSDT"
g_interval = "30m" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
g_histDays = 0  # 30 10 60 84 120 
g_limit = 998 # 1500  499 998

ukdfHist = getDayUKline(symbol=g_symbol, interval=g_interval, startDate="20220701") # , endDate="0", histDays=0, limit=499
# print(ukdfHist.shape)

conn = sqlite3.connect(g_dbfile)
# ##df_uk = pd.read_sql(' select * from uk30m ', conn) 
ukdfHist.to_sql('uk30m', con=conn, if_exists='append', index=False) #,
conn.close()

ukdfHist.to_pickle(f"{g_worthDir}/uk30mdf.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk30mdf.pkl" )


In [ ]:
# 2.查询指定起止日期行情 getDayUKline() 4h
g_symbol = "BTCUSDT"
g_interval = "4h" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
g_histDays = 0  # 30 10 60 84 120 
g_limit = 998 # 1500  499 998

ukdfHist = getDayUKline(symbol=g_symbol, interval=g_interval, startDate="20210101") # , endDate="0", histDays=0, limit=499
# Amt,bidAmt,bidVol,close,closeDateTime,closeTime,high,low,numTrade,open,openDateTime,openTime,vol
# print(ukdfHist.shape)

conn = sqlite3.connect(g_dbfile)
# ##df_uk = pd.read_sql(' select * from uk4h ', conn) 
ukdfHist.to_sql( f"uk{g_interval}", con=conn, if_exists='append', index=False) #,
conn.close()

ukdfHist.to_pickle(f"{g_worthDir}/uk{g_interval}df.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk30mdf.pkl" )


In [ ]:
# 3.查询指定天数行情 histUKline()  4h
symbol = "BTCUSDT"
interval = "4h" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
histDays = 2  # 30 10 60 84 120 
limit = 998 # 1500  499 998
wid = 42

ukdf = histUKline(interval=interval, histDays=histDays, limit=498)
print(ukdf.shape)
# print(ukdf)
# print(f'ukdf.iloc[:5,:] :\n{ukdf.iloc[:5,:]}')
# print(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')

# ukRet = continuousKlines(pair=symbol, interval="5m", limit=3 )
# kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+g_locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])
# print(kdf)

# t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
# t1.start()
# stop_thread(thread)

conn = sqlite3.connect(g_dbfile)
# ##df_uk = pd.read_sql(' select * from uk4h ', conn) 
ukdf.to_sql('uk4h', con=conn, if_exists='append', index=False) #,
conn.close()

ukdf.to_pickle(f"{g_worthDir}/uk4hdf.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk4hdf.pkl" )

# nohup python3 -u main.py -n testStrategy -s 8808 -c 29090 -X BTCUSDT -p 10m -w 42 -d 84 -t 1000000  >> log.txt 2>&1  &


In [ ]:
# 4 查询指定天数行情 histUKline() 5m
symbol = "BTCUSDT"
interval = "5m" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
histDays = 30  # 30 10 60 84 120 
limit = 998 # 1500  499 998
wid = 42

ukdf = histUKline(interval=interval, histDays=histDays, limit=498)
print(ukdf.shape)
# print(ukdf)
print(f'ukdf.iloc[:5,:] :\n{ukdf.iloc[:5,:]}')
print(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')


# conn = sqlite3.connect(dbfile)
# ##df_uk = pd.read_sql(' select * from uk4h ', conn) 
# ukdf.to_sql('uk4h', con=conn, if_exists='append', index=False) #,
# conn.close()

ukdf.to_pickle(f"{g_worthDir}/uk5mdf.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk5mdf.pkl" )

# nohup python3 -u main.py -n testStrategy -s 8808 -c 29090 -X BTCUSDT -p 10m -w 42 -d 84 -t 1000000  >> log.txt 2>&1  &


In [ ]:
# 5 查询指定年份行情 getDayUKline() 
g_symbol = "BTCUSDT"
g_interval = "5m" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
g_histDays = 0  # 30 10 60 84 120 
g_histYear = '2021' # 2021 2022
g_limit = 998 # 1500  499 998

ukdfHist = getDayUKline(symbol=g_symbol, interval=g_interval, startDate=f"{g_histYear}0101" ) # , endDate="0", histDays=0, limit=499
# Amt,bidAmt,bidVol,close,closeDateTime,closeTime,high,low,numTrade,open,openDateTime,openTime,vol
# print(ukdfHist.shape)

conn = sqlite3.connect(g_dbfile)
# ##df_uk = pd.read_sql(' select * from uk4h ', conn) 
ukdfHist.to_sql( f"uk{g_interval}_{g_histYear}", con=conn, if_exists='append', index=False) #,
conn.close()

ukdfHist.to_pickle(f"{g_worthDir}/uk{g_interval}_{g_histYear}df.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk30mdf.pkl" )


In [ ]:
# 6 补全/查询指定日期行情 getDayUKline() 
g_symbol = "BTCUSDT"
g_interval = "5m" # period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
g_histDays = 0  # 30 10 60 84 120 
g_limit = 998 # 1500  499 998

df_uk = pd.DataFrame()
startDate = 0

df_uk = pd.read_pickle(f"{g_worthDir}/uk{g_interval}df.pkl" )
startDate = df_uk.tradeDate.iloc[-1]

print(f"{df_uk.shape }")
print(f"df_uk.head(2): \n{df_uk.head(2) }")
print(f"df_uk.tail(2): \n{df_uk.tail(2) }")

ukdfHist = getDayUKline(symbol=g_symbol, interval=g_interval, startDate=startDate) # , endDate="0", histDays=0, limit=499
# print(ukdfHist.shape)

kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+g_locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close,'high': rs.high,'low': rs.low,'vol': rs.vol, 'amt': rs.Amt} for _, rs in ukdfHist.iterrows() ])

kdf['close'] = kdf['close'].astype(float)
kdf['pct'] = kdf['close']/kdf['close'].shift(1)-1  #涨跌幅
kdf.fillna(0, inplace=True)

# df_uk.sort_values(by='closeSec') # 按 closeSec列数据升序排列  # 降序 , ascending=False
df_uk = pd.concat([df_uk,kdf],ignore_index=True)
df_uk.drop_duplicates(subset=["closeSec"], keep="last", inplace=True, ignore_index=True)
df_uk['pct'] = df_uk['close']/df_uk['close'].shift(1)-1  #涨跌幅

# conn = sqlite3.connect(g_dbfile)
# # ##df_uk = pd.read_sql(' select * from uk5m ', conn) 
# ukdfHist.to_sql('uk5m', con=conn, if_exists='append', index=False) #,
# conn.close()

df_uk.to_pickle(f"{g_worthDir}/uk{g_interval}df.pkl" )
# ##df_uk = pd.read_pickle(f"{g_worthDir}/uk5mdf.pkl" )

print(f"{df_uk.shape }")
print(f"df_uk.head(2): \n{df_uk.head(2) }")
print(f"df_uk.tail(2): \n{df_uk.tail(2) }")

df1 = df_uk[df_uk.tradeDate == startDate]
print(f"{df1.shape }")
print(f"df1.head(2): \n{df1.head(2) }")
print(f"df1.tail(2): \n{df1.tail(2) }")


In [ ]:
# 7 合并指定日期行情 getDayUKline() uk5m_2021df

uk5mFolder = f"/workspaces/jupyter/tmp" 
# /workspaces/jupyter/tmp/uk5m_2021df.pkl /workspaces/backup/bigdata
uk5mFile = f"{uk5mFolder}/uk5m_2021df.pkl"
df_uk = pd.read_pickle(f"{uk5mFile}" )

# uk_df = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+g_locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+g_locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close,'high': rs.high,'low': rs.low,'vol': rs.vol, 'amt': rs.Amt} for _, rs in df_uk.iterrows() ])

kdf = pd.read_pickle(f"{g_worthDir}/uk5mdf.pkl" )

# df_uk.sort_values(by='closeSec') # 按 closeSec列数据升序排列  # 降序 , ascending=False
df_uk = pd.concat([df_uk,kdf],ignore_index=True)
df_uk.drop_duplicates(subset=["closeSec"], keep="last", inplace=True, ignore_index=True)
kdf['close'] = kdf['close'].astype(float)
df_uk['pct'] = df_uk['close']/df_uk['close'].shift(1)-1  #涨跌幅

df_uk.to_pickle(f"{uk5mFile}" )

print(f"{df_uk.shape }")
print(f"df_uk.head(2): \n{df_uk.head(2) }")
print(f"df_uk.tail(2): \n{df_uk.tail(2) }")

In [15]:
# print( f"{ ukdfHist.head(2) }" )

# print( f"{ ukdfHist.tail(2) }" )

# ukdfHist.head(2)
# ukdfHist.tail(2)

# df_uk = pd.read_pickle(f"{g_worthDir}/uk30mdf.pkl" )
# df_uk.close.tail(2)

# df_uk = pd.read_pickle(f"{g_worthDir}/uk4hdf.pkl" )
# df_uk.closeDateTime.tail(2)

df_uk = pd.read_pickle(f"{g_worthDir}/uk5mdf.pkl" )
# df_uk.closeDateTime.tail(2)

print(f"{df_uk.shape }")
print(f"df_uk.head(2): \n{df_uk.head(2) }")
print(f"df_uk.tail(2): \n{df_uk.tail(2) }")

startDate = df_uk.tradeDate.iloc[-1]

print(f"{type(startDate)}, {startDate = }")


(18861, 11)
df_uk.head(2): 
  tradeDate openTime closeTime    closeSec     open    close     high      low        vol           amt       pct
0  20221024   000000    000459  1666541099  19335.0  19335.0  19335.0  19200.2  18.882999  3.636677e+05       NaN
1  20221024   000500    000959  1666541399  19334.0  19339.0  19339.0  19201.0  53.950001  1.041028e+06  0.000207
df_uk.tail(2): 
      tradeDate openTime closeTime    closeSec     open    close     high      low         vol           amt       pct
18859  20221228   113500    113959  1672198799  16824.8  16658.8  16824.8  16647.0  468.325989  7.811034e+06  0.000228
18860  20221228   114000    114459  1672199099  16652.5  16828.8  16828.8  16646.2  205.018997  3.424889e+06  0.010205
<class 'str'>, startDate = '20221228'
